In [62]:
import pandas as pd
import dask
import dask.dataframe as dd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [325]:
files = [glob(f'data/tree2_090518/*/{QEP}*.txt') for QEP in ['QEP01', 'QEP02']]  
files[0][:5],  len(files[0])

(['data/tree2_090518/part_1/QEP011681_06-20-20-248.txt',
  'data/tree2_090518/part_1/QEP011681_06-20-35-230.txt',
  'data/tree2_090518/part_1/QEP011681_06-20-50-230.txt',
  'data/tree2_090518/part_1/QEP011681_06-21-05-231.txt',
  'data/tree2_090518/part_1/QEP011681_06-21-20-231.txt'],
 4019)

In [326]:
import datetime_glob
import pytz


def parse_date(s):
    matcher = datetime_glob.Matcher(pattern='data/tree?_%m%d%y/part_?/QEP*_%H-%M-%S-*.txt')
    tz = pytz.timezone('US/Eastern')
    t = matcher.match(path=s)
    if t is None:
        raise ValueError(s)
    else:
        return tz.localize(t.as_datetime())


def read_file(path):
    instrument = ''.join(path.split('/')[-1][:9])
    time = parse_date(path)
    df = dd.read_csv(path, header=4, names=['wl', 'val'], sep='\t')
    return xr.DataArray(
        df['val'].to_dask_array(lengths=True),
        dims=('wavelength'), 
        coords={'time': time, 'wavelength': df['wl'].to_dask_array()}, name=instrument)

In [328]:
ds = xr.merge([xr.concat([read_file(f) for f in files[k]], dim='time') for k in [0,1]])

KeyboardInterrupt: 

In [ ]:
ds